In [1]:
!pip install dspy pyaxis

In [1]:
import requests
import json
from pyaxis import pyaxis
from bs4 import BeautifulSoup
import pandas as pd
import dspy
import asyncio
from google.colab import userdata

MAKSTAT_URL = "https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/search"

lm = dspy.LM('openai/gpt-4o-mini', api_key=userdata.get('OpenAI_API'))
dspy.configure(lm=lm)

In [17]:
from IPython.display import display, HTML
import sys
import textwrap

def keyword_search(keyword: str) -> dict[str, list[str]]:
    """A function that performs keyword search (in Macedonian) to retrieve a list of relevant tables from the National Statistics Office.
    """
    session = requests.Session()

    # Step 1: initial GET
    r = session.get(MAKSTAT_URL, params={"searchquery": keyword})
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Step 2: collect hidden inputs
    data = {}
    for inp in soup.select("input[type=hidden]"):
        if inp.get("name"):
            data[inp["name"]] = inp.get("value", "")

    # Step 3: add your desired options
    data["ctl00$ContentPlaceHolderMain$pxSearch$txtSearch"] = keyword
    data["ctl00$ContentPlaceHolderMain$pxSearch$cmdSearch"] = "Пребарај"
    data["ctl00$ContentPlaceHolderMain$searchOptions"] = "select"   # "Пребарај само"
    data["ctl00$ContentPlaceHolderMain$chkTitle"] = "on"            # "Наслов на табелата"

    # Step 4: POST back
    r2 = session.post(MAKSTAT_URL, data=data)
    r2.raise_for_status()

    # wrap in soup
    soup2 = BeautifulSoup(r2.text, "html.parser")

    rows = soup2.select("tr")
    results = {"titles": [], "links": [], "published": []}

    for row in rows:
        a = row.select_one("td.searchCellTable a")
        pub = row.select_one("td.searchCellPublished")
        if a and pub:
            results["titles"].append(a.get_text(strip=True))
            results["links"].append("https://makstat.stat.gov.mk" + a["href"])
            results["published"].append(pub.get_text(strip=True))

    return results

def get_table(url: str) -> pd.DataFrame:
    """A function that retrieves the full contents of the table in markdown. The table might be long, so use with caution only when necessary.

    Make sure the URL you provide as an argument is exactly what you see in the keyword search results!
    """
    API_BASE_URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/"
    API_suffix = url.replace("MakStat__", "").replace("__", "/").split("pxweb/")[1]
    TABLE_URL = API_BASE_URL + API_suffix
    query = {
      "query": [],
      "response": {
        "format": "px"
      }
    }
    response = requests.post(TABLE_URL, json=query)
    if response.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {response.status_code}")

    with open("result.px", 'wb') as f:
        f.write(response.content)

    #try:
    result = pyaxis.parse('result.px', encoding='utf-8')
    #except UnicodeDecodeError:
    #    print("UTF-8 decoding failed, trying ISO-8859-5")
    #    result = pyaxis.parse('result.px', encoding='ISO-8859-5')


    table_df = pd.DataFrame(result['DATA'])

    return table_df.to_markdown()


react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[keyword_search, get_table],
    max_iters=5
)

#result = react_agent(question="Колку табели можеш да најдеш поврзани со трговија?")

stream_listeners = [
    # dspy.ReAct has a built-in output field called "next_thought".
    dspy.streaming.StreamListener(signature_field_name="next_thought", allow_reuse=True),
]
stream_react = dspy.streamify(react_agent, stream_listeners=stream_listeners)

async def read_output_stream(width=80):
    output = stream_react(question="Колку табели можеш да најдеш поврзани со трговија?")
    return_value = None
    async for chunk in output:
        if isinstance(chunk, dspy.streaming.StreamResponse):
            #print(chunk.chunk)
            raw = getattr(chunk, "chunk", chunk)
            if isinstance(raw, bytes):
                raw = raw.decode("utf-8", "replace")
            # remove leading newlines (common when server sends "\n" at chunk start)
            raw = raw.lstrip("\n")
            # if you also want to strip internal newlines, use replace:
            # raw = raw.replace("\n", " ")
            sys.stdout.write(raw)
            sys.stdout.flush()
        elif isinstance(chunk, dspy.Prediction):
            return_value = chunk
    return return_value

print(await read_output_stream())

Request successful!

Multilingual PX file


Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb2 in position 220: invalid start byte



Имам проблем со добивањето на деталите за втората табела. Можеби ќе треба да направам ново пребарување за табели поврзани со трговија за да видиме дали можам да најдам друга табела или да добијам информации од друга извор.Не можам да добијам деталите за втората табела поради проблем со кодирањето. Можеби ќе треба да завршам со задачата, бидејќи веќе имам информации за првата табела и не можам да добијам дополнителни податоци.

Prediction(
    trajectory={'thought_0': 'Треба да направам пребарување за табели поврзани со трговија за да ги добијам 
потребните информации.', 'tool_name_0': 'keyword_search', 'tool_args_0': {'keyword': 'трговија'}, 'observation_0': 
{'titles': ['Е-трговија кај деловните субјекти од нефинансискиот сектор, со 10 или повеќе вработени (%)', 'Промет 
во трговијата со моторни возила, трговија на големо и на мало, по НКД Рев. 2, по месеци'], 'links': 
['https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Enteco
m_ml.px/', 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__VnatresnaTrgovija__VTBazna2021/125_Vt_mk_11_TrgDej_202
1_ml.px/'], 'published': ['08.4.2025', '01.9.2025']}, 'thought_1': 'Најдов две табели поврзани со трговија. Сега 
можам да ги добијам деталите за секоја од нив.', 'tool_name_1': 'get_table', 'tool_args_1': {'url': 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Entecom
_ml.px/'}, 'observation_1': '|    | Деловни субјекти (10+) кои имале е-трговија                                    
|   Година | DATA   
|\n|---:|:---------------------------------------------------------------------------------------------------------
-|---------:|:-------|\n|  0 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани 
мрежи (е-купување)        |     2006 | 14.6   |\n|  1 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2007 | 7.9    |\n|  2 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2008 | 9.8    |\n|  3 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2009 | 7.1 
|\n|  4 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)       
|     2010 | 6.8    |\n|  5 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2011 | 8.1    |\n|  6 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2012 | 5.1    |\n|  7 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2013 | 4.6    |\n|  8 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2014 | 5.0 
|\n|  9 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)       
|     2015 | 5.9    |\n| 10 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2016 | 5.6    |\n| 11 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2017 | 5.8    |\n| 12 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2018 | "..."  |\n| 13 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2019 | 
"..."  |\n| 14 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)
|     2020 | "..."  |\n| 15 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2021 | "..."  |\n| 16 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2022 | "..."  |\n| 17 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2023 | "..."  |\n| 18 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2024 | 
"..."  |\n| 19 | Деловни субјекти кои примиле нарачки преку интернет или други компјутерски посредувани мрежи 
(е-про

In [18]:
result = react_agent(question="Пребарај табели поврзани со трговија и дај ми го линкот за табелата за моторни возила?")
print(result.answer)
print("Tool calls made:", result.trajectory)

Request successful!

Multilingual PX file


[Линк до табелата за регистрирани патни моторни и приклучни 
возила](https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_RegVo
zila_ml.px/)

Tool calls made:
{
    'thought_0': 'Треба да извршам пребарување за табели поврзани со трговија, особено за моторни возила. Ова ќе ми
помогне да најдам линк до соодветната табела.',
    'tool_name_0': 'keyword_search',
    'tool_args_0': {'keyword': 'трговија моторни возила'},
    'observation_0': {
        'titles': [
            'Промет во трговијата со моторни возила, трговија на големо и на мало, по НКД Рев. 2, по месеци',
            'Просечна старост на патните моторни возила, 2003-2024',
            'Регистрирани патни моторни и приклучни возила , по години, по региони',
            'Регистрирани патни моторни и приклучни возила според видот и годината на производство, состојба 
31.12.2024',
            'Е-трговија кај деловните субјекти од нефинансискиот сектор, со 10 или повеќе вработени (%)',
            'Моторен сообраќај по категории на возила и вид на сообраќај во 2005, 2010 и 2015 година',
            'Дистрибуција на моторниот сообраќај по категории на возила според пописите во 2005, 2010 и 2015 
година',
            'Регистрирани патни моторни и приклучни превозни средства, по општини, по години, број',
            'Регистрирани патни моторни и приклучни превозни средства, прва регистрација, по општини, по години, 
број',
            'Регистрирани патни моторни превозни средства по гориво, по општини, по години, број'
        ],
        'links': [
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__VnatresnaTrgovija__VTBazna2021/125_Vt_mk_1
1_TrgDej_2021_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/225_Trans_av
erageage_vehicle_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_
RegVozila_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/250_Trans_go
dina_proizv_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_
Mk_12Entecom_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__PopisEpatenSoobrakaj/550_Trans_
T2_MK_Motsoobr_mk.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__PopisEpatenSoobrakaj/575_Trans_
Mk_Т41_mk.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/150_Trans_re
gi_veh_by_type_mun_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/200_Trans_pr
v_regi_voz_po_vid_OPST_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/275_Trans_Re
g_voz_Gorivo_ml.px/'
        ],
        'published': [
            '01.9.2025',
            '08.8.2025',
            '07.8.2025',
            '08.8.2025',
            '08.4.2025',
            '21.11.2017',
            '21.11.2017',
            '12.8.2025',
            '12.8.2025',
            '06.8.2025'
        ]
    },
    'thought_1': 'Сега имам список на табели поврзани со моторни возила. Треба да го најдам линкот за табелата која
е најрелевантна за моторни возила. Најсоодветната табела изгледа да е "Регистрирани патни моторни и приклучни 
возила , по години, по региони". Ќе ја повикам функцијата get_table за да добијам целосна содржина на табелата.',
    'tool_name_1': 'get_table',
    'tool_args_1': {
        'url': 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_RegVozila_ml
.px/'
    },
    'observation_1': '|      | Регион               |   Година | Вид на возило       |   DATA 
|\n|-----:|:---------------------|---------:|:--------------------|-------:|\n|    0 | Република Македонија |     
2007 | Вкупно              | 287222 |\n|    1 | Република Македонија |     2007 | мотоцикли           |   4437 |\n|
2 | Република Македонија |     2007 | патнички автомоб

In [19]:
result = react_agent(question="Прво најди ги табелите поврзани со образование, и потоа извлечи ја најинтересната и покажи ми ја.")
print(result.answer)
print("Tool calls made:", result.trajectory)

Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 222: invalid continuation byte



Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 239: invalid continuation byte



Request successful!

Multilingual PX file


| Возраст   | Година | Степен на образование   | Пол    | DATA   |
|-----------|--------|------------------------|--------|--------|
| Вкупно    | 2017   | Вкупно                  | Вкупно | 154646 |
| Вкупно    | 2017   | Вкупно                  | Мажи   | 75560  |
| Вкупно    | 2017   | Вкупно                  | Жени   | 79086  |
| Вкупно    | 2017   | Од I до IV одделение    | Вкупно | "-"    |
| Вкупно    | 2017   | Од V до VIII одделение  | Вкупно | 2333   |
| Вкупно    | 2017   | Средно образование      | Вкупно | 72061  |
| Вкупно    | 2017   | Више образование        | Вкупно | 744    |
| Вкупно    | 2017   | Високо образование      | Вкупно | 72685  |
| Вкупно    | 2017   | Магистратура и докторат | Вкупно | 6823   |
| Вкупно    | 2018   | Вкупно                  | Вкупно | 152134 |
| Вкупно    | 2018   | Од V до VIII одделение  | Вкупно | 4159   |
| Вкупно    | 2018   | Средно образование      | Вкупно | 72355  |
| Вкупно    | 2018   | Више образование        | Вкупно | 1135   |
| Вкупно    | 2018   | Високо образование      | Вкупно | 67690  |
| Вкупно    | 2018   | Магистратура и докторат | Вкупно | 6529   |
| Вкупно    | 2019   | Вкупно                  | Вкупно | 154575 |
| Вкупно    | 2019   | Од V до VIII одделение  | Вкупно | 2253   |
| Вкупно    | 2019   | Средно образование      | Вкупно | 76587  |
| Вкупно    | 2019   | Више образование        | Вкупно | 1805   |
| Вкупно    | 2019   | Високо образование      | Вкупно | 67430  |
| Вкупно    | 2019   | Магистратура и докторат | Вкупно | 6274   |
| Вкупно    | 2020   | Вкупно                  | Вкупно | 150375 |
| Вкупно    | 2020   | Од V до VIII одделение  | Вкупно | 2735   |
| Вкупно    | 2020   | Средно образование      | Вкупно | 74992  |
| Вкупно    | 2020   | Више образование        | Вкупно | 1612   |
| Вкупно    | 2020   | Високо образование      | Вкупно | 64082  |
| Вкупно    | 2020   | Магистратура и докторат | Вкупно | 6811   |
| Вкупно    | 2021   | Вкупно                  | Вкупно | 158078 |
| Вкупно    | 2021   | Од V до VIII одделение  | Вкупно | 3791   |
| Вкупно    | 2021   | Средно образование      | Вкупно | 76418  |
| Вкупно    | 2021   | Више образование        | Вкупно | 1818   |
| Вкупно    | 2021   | Високо образование      | Вкупно | 69617  |
| Вкупно    | 2021   | Магистратура и докторат | Вкупно | 6434   |
| Вкупно    | 2022   | Вкупно                  | Вкупно | 158364 |
| Вкупно    | 2022   | Од V до VIII одделение  | Вкупно | 3448   |
| Вкупно    | 2022   | Средно образование      | Вкупно | 77590  |
| Вкупно    | 2022   | Више образование        | Вкупно | 1557   |
| Вкупно    | 2022   | Високо образование      | Вкупно | 69621  |
| Вкупно    | 2022   | Магистратура и докторат | Вкупно | 6096   |
| Вкупно    | 2023   | Вкупно                  | Вкупно | 152703 |
| Вкупно    | 2023   | Од V до VIII одделение  | Вкупно | 2444   |
| Вкупно    | 2023   | Средно образование      | Вкупно | 72463  |
| Вкупно    | 2023   | Више образование        | Вкупно | 802    |
| Вкупно    | 2023   | Високо образование      | Вкупно | 69814  |
| Вкупно    | 2023   | Магистратура и докторат | Вкупно | 5636   |
| Вкупно    | 2024   | Вкупно                  | Вкупно | 151015 |
| Вкупно    | 2024   | Од V до VIII одделение  | Вкупно | 3167   |
| Вкупно    | 2024   | Средно образование      | Вкупно | 72631  |
| Вкупно    | 2024   | Више образование        | Вкупно | nan    |
| Вкупно    | 2024   | Високо образование      | Вкупно | 67582  |
| Вкупно    | 2024   | Магистратура и докторат | Вкупно | 7146   |

Tool calls made:
{
    'thought_0': 'Морам да започнам со пребарување на табелите поврзани со образование. Ова ќе ми помогне да ги 
пронајдам релевантните информации за следниот чекор.',
    'tool_name_0': 'keyword_search',
    'tool_args_0': {'keyword': 'образование'},
    'observation_0': {
        'titles': [
            'ЦОР 4. Квалитетно образование',
            'Показатели од половите статистики - Образование',
            'Работоспособно население кое посетува редовно образование, според степенот на образование и возраста, 
по години',
            'Работоспособно население кое посетува редовно образование, според пол и полето на образование, по 
години',
            'Работоспособно население кое посетува редовно образование, според степенот на образование и возраста, 
по години, архивирани податоци',
            'Работоспособно население кое посетува редовно образование, според пол и полето на образование, по 
години, архивирани податоци',
            'Нецелосна програма за формално образование според економски статус и причини',
            'Дали нивото на завршено образование одговара на претходното работно место',
            'Лица кои посетуваат образование и доживотно учење по занимање',
            'Пречки за учество во образование и доживотно учење, по пол',
            'Пречки за учество во образование и доживотно учење, по возраст',
            'Работоспособно население според економската активност, пол и учеството во формалното (редовно) 
образование, степенот на образование, по години',
            'Работоспособно население според економската активност, пол и учеството во формалното (редовно) 
образование, степенот на образование, по години, архивирани податоци',
            'Вкупно резидентно население на возраст од 15+ години според полето на образование во највисокиот 
степен на образование, пол и возраст, по општини, Попис, 2021',
            'Вкупно резидентно население на возраст од 5+ години во формалното (редовно) образование, според 
степенот на образование, пол и возраст, по општини, Попис, 2021',
            'Вкупно резидентно население на возраст од 15+ години во формалното (редовно) образование според  поле 
на образование, пол и возраст, по општини, Попис, 2021',
            'Стапки на запишување во основно образование, по пол, по учебна година',
            'Стапки на запишување во средно образование, по пол, по учебна година',
            'Вработени (број на лица) во истражувачко развојна дејност според степенот на образование и секторот на
извршување',
            'Лица со низок степен на образование, по возрасни групи, по години',
            'Статус на учество во програми за формално образование според економски статус',
            'Степен до кој нивото на завршено образование одговара на главното работно место',
            'Степен до кој завршеното образование одговара на барањата на тековната работа',
            'Степен до кој професијата од завршеното образование одговара на барањата на работното место',
            'Лица кои посетуваат образование и доживотно учење според возраста, урбан и рурален дел',
            'Лица кои посетуваат образование и доживотно учење според полот и школската подготовка',
            'Лица кои посетуваат образование и доживотно учење според возраста и школската подготовка',
            'Лица кои посетуваат образование и доживотно учење според економската активност, по возараст',
            'Лица кои посетуваат образование и доживотно учење според економската активност, урбан и рурален дел',
            'Лица кои посетуваат образование и доживотно учење по сектори на дејности',
            'Учество во неформално образование, според видот, полето и бројот на образовните активности',
            'Учество во неформално образование, според видот на образовната активност и субјектот кој ја 
спроведува',
            'Просечен број на часови поминати во образование и обука, по возраст и по пол',
            'Просечен број на часови поминати во образование и о

# Build eval dataset